## clear sky irradiance

In [43]:
from pvlib import irradiance
# import rasterio
import numpy as np
from glob import glob
from scipy import stats
import os
import sys
from pvlib.location import Location

location = Location(latitude=44.475883, longitude=-73.212074)
#inputs times (pd.datetimeindex)
#pressure (none/supplied) if none uses atmosphere.alt2pres() and self.altitude
#temp

In [25]:
import pvlib as pvlib
import pandas as pd

In [35]:
import datetime

In [62]:
now = datetime.datetime.now()

In [63]:
df = pd.to_datetime(now)

In [64]:
df

Timestamp('2020-04-14 11:25:18.396244')

In [65]:
df = location.get_solarposition(times = df, temperature = 14)
df

,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
2020-04-14 11:25:18.396244,77.432433,77.504813,12.567567,12.495187,88.658081,-0.174516


# Relationship Between Sources of Irradiance

GHI = DNI * cos(SZA) + DHI <br> <br>

GHI = Global Horizontal Irradiance <br>
SZA =e Solar Zenith Angle <br>
DHI = Diffuse Horizontal Irradiance <br>

In [72]:
# Estimate DNI and DHI from GHI using Erbs model
erbs = irradiance.erbs
#INPUTS
ghi = 100
zenith = df['zenith']
datetime_or_doy = df.index
min_cos_zenith = None
max_zenith = None

# https://www.sciencedirect.com/science/article/abs/pii/0038092X82903024

In [73]:
datetime_or_doy

DatetimeIndex(['2020-04-14 11:25:18.396244'], dtype='datetime64[ns]', freq=None)

In [77]:
erbs(ghi=ghi, zenith=zenith, datetime_or_doy = datetime_or_doy)

,dni,dhi,kt
2020-04-14 11:25:18.396244,39.712324,91.407937,0.340642


In [75]:
# Estimate Direct Normal Irradiance from Global Horizontal Irradiance using the DISC model
disc = irradiance.disc
#INPUTS
ghi = 100
zenith = df['zenith']
datetime_or_doy = df.index
pressure = None
min_cos_zenith = None
max_zenith = None
max_airmass = None

# https://www.nrel.gov/grid/solar-resource/disc.html

In [76]:
disc(ghi=ghi, solar_zenith=zenith, datetime_or_doy = datetime_or_doy)

,dni,kt,airmass
2020-04-14 11:25:18.396244,89.919055,0.339672,4.52752


# model one day

In [ ]:
erbs(ghi=ghi, zenith=zenith, datetime_or_doy = datetime_or_doy)